In [ ]:
from cng.utils import *
from cng.h3 import *
from ibis import _
import ibis.selectors as s
import os
from osgeo import gdal
from minio import Minio
import streamlit 
from datetime import timedelta
import geopandas as gpd
import re

# Get signed URLs to access license-controlled layers
key = st.secrets["MINIO_KEY"]
secret = st.secrets["MINIO_SECRET"]
client = Minio("minio.carlboettiger.info", key, secret)

con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
endpoint = os.getenv("AWS_S3_ENDPOINT", "minio.carlboettiger.info")
duckdb_install_h3()

set_secrets(con)


In [ ]:
%%time
#Wall time: 9min 39s
parquet_file = f's3://public-ca30x30/ca30x30_cbn_v3.parquet'
geobuf_file = 'ca30x30_cbn_v3.fgb'
ca_geo = con.read_parquet(parquet_file).mutate(acres = _.acres.round(4))
ca_geo.select(~s.startswith(["mean_", "pct_"])).execute().set_crs('epsg:4326').to_file(geobuf_file)

In [ ]:
%%time 
#Wall time: 20min 7s
pmtiles_file = 'ca30x30_cbn_v3.pmtiles'
pmtiles = f's3://public-ca30x30/{pmtiles_file}'
source_layer_name = re.sub(r'\W+', '', os.path.splitext(os.path.basename(pmtiles_file))[0])

options =[f'--layer={source_layer_name}',
            '-z12',
            '--drop-rate=0',  
            '--extend-zooms-if-still-dropping',  
            '--no-feature-limit',
            '--no-tile-size-limit',
            '--no-tiny-polygon-reduction',
            '--no-simplification-of-shared-nodes'
         ]
new_pmtiles = to_pmtiles(geojson_file, pmtiles_file, options = options )
s3_cp(new_pmtiles,pmtiles)

In [ ]:
pmtiles = f'https://minio.carlboettiger.info/public-ca30x30/{pmtiles_file}'

paint = {
    "fill-color": "#d09514",
    "fill-opacity": 0.6,
}
style = {'version': 8, 'sources': 
         {'ca': {'type': 'vector', 'url': f'pmtiles://{pmtiles}'}},
         'layers': [{'id': 'ca30x30', 'source': 'ca',
                     'source-layer': 'ca30x30_cbn_v3', 
                     'type': 'fill', 
                     'paint': {'fill-color': {'property': 'status',
                                              'type': 'categorical', 
                                              'stops': [['30x30 Conservation Area', "#56711f"],
                                                        ['Other Conservation Area', "#b6ce7a"],
                                                        ['Public or Unknown Conservation Area', "#e5efdb"],
                                                        ['Non-Conservation Area', "#e1e1e1"]],
                                              'default': '#FFFFFF'}, 'fill-opacity': 0.8}}]}


import leafmap.foliumap as leafmap

m = leafmap.Map(center=[35, -100], zoom=5, style="positron", fullscreen_control=True)

m.add_pmtiles(pmtiles, style, zoom_to_layer = True)
m

